In [7]:
import numpy as np
%matplotlib inline

## Data load

In [8]:
data = open('1342-0.txt', 'r').read()
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
char_to_ix = { ch:i for i, ch in enumerate(chars) }
ix_to_char = { i:ch for i, ch in enumerate(chars) }
print(char_to_ix)

{'U': 0, 'G': 1, 'z': 2, 'D': 3, '0': 4, 'o': 5, 'i': 6, '1': 7, '-': 8, 'V': 9, 'X': 10, '(': 11, 's': 12, 'v': 13, 'h': 14, ':': 15, '5': 16, 'B': 17, 'Q': 18, 'I': 19, '8': 20, 'd': 21, 'S': 22, '%': 23, 'O': 24, "'": 25, '“': 26, 'b': 27, 'k': 28, 't': 29, 'f': 30, '/': 31, 'r': 32, 'Z': 33, 'u': 34, ';': 35, ' ': 36, '.': 37, '*': 38, 'J': 39, 'x': 40, 'H': 41, 'y': 42, 'g': 43, 'L': 44, 'j': 45, '6': 46, '2': 47, '\n': 48, '?': 49, 'K': 50, 'e': 51, '#': 52, 'F': 53, '!': 54, ']': 55, 'E': 56, 'w': 57, 'q': 58, 'n': 59, '$': 60, 'c': 61, ')': 62, 'l': 63, 'R': 64, '7': 65, 'p': 66, '_': 67, 'P': 68, 'M': 69, '@': 70, '”': 71, 'a': 72, ',': 73, '9': 74, 'T': 75, '3': 76, 'C': 77, '\ufeff': 78, 'W': 79, 'm': 80, 'Y': 81, '[': 82, 'N': 83, 'A': 84, '4': 85}


In [14]:
vector_for_char_a = np.zeros((vocab_size, 1))
vector_for_char_a[char_to_ix['a']] = 1

(86, 1)


### hyperparameters

In [15]:
hidden_size = 100
seq_length = 25
learning_rate = 1e-1

## RNN

### Backward step computation for one timestep

I) $$ \frac{\partial (tanh(x))}{\partial (x)} = 1 - tanh^2(x)$$

II) $$ \frac{\partial (Ux_t + Ws_{t-1} + b)}{\partial (Ux_t)} = 1$$
$$ \frac{\partial (Ux_t + Ws_{t-1} + b)}{\partial (Ws_{t-1})} = 1$$
$$ \frac{\partial (Ux_t + Ws_{t-1} + b)}{\partial (b)} = 1$$


III.a) $$ \frac{\partial (Ws_{t-1})}{\partial (W)} = s_{t-1}$$
$$ \frac{\partial (Ws_{t-1})}{\partial (s_{t-1})} = W$$

III.b) $$ \frac{\partial (Ux_t)}{\partial (U)} = x_t$$
$$ \frac{\partial (Ux_t)}{\partial (x_t)} = U$$


In [ ]:
def cross_entropy(x, y):
    '''
    x: input in a one hot encoding
    y: index encoding
    '''
    return -np.log(x[y, 0])

def cross_entropy_derivative(y_predicted, y):
    '''
    y_predicted: input in a one hot encoding
    y: index encoding
    '''
    y_predicted[y] -= 1
    return y_predicted

def softmax(x):
    exps = np.exp(x)
    return exps / np.sum(exps)


In [29]:
class RNN():
    def __init__(self, feature_length, 
                 loss_function, loss_function_derivative, 
                 activation_function=np.tanh, activation_function_derivative=
                 hidden_size=100, seq_length = 25, learning_rate = 1e-1):
        
        _variace = 0.01
        self.U = np.random.randn(hidden_size, vocab_size) * _variace # to input
        self.W = np.random.randn(hidden_size, hidden_size) * _variace # to recurrent
        self.V = np.random.randn(vocab_size, hidden_size) * _variace # to output
        
        b_s = np.zeros((hidden_size, 1)) # hidden bias
        b_y = np.zeros((vocab_size, 1)) # output bias
        
        self.learning_rate = learning_rate
        self.hidden_size = hidden_size
        self.seq_length = seq_length
        self.feature_length = feature_length
        
        self.s = np.zeros((seq_length, hidden_size))
        self.s[-1,:] = np.zeros(hidden_size)
    
    def _step_forward(self, x, prev_s):
        next_s = self.activation_function(np.dot(self.U, x) + np.dot(self.W, prev_s) + self.b_s)
        y_predicted = np.dot(self.V, next_s) + self.b_y
        probability_predicted = softmax(y_predicted)
        return next_s, y_predicted, probability_predicted
    
    def forward(self, x, y):
        '''
        x: input in a one hot encoding
        y: index encoding
        '''
        s, y_predicted, probability_predicted = {}, {}, {}
        loss = 0
        for t in range(self.seq_length):
#             self.s[t] = self.activation_function(np.dot(self.U, x[t]) + np.dot(self.W, s[t - 1]) + self.b_s)
#             y_predicted[t] = np.dot(self.V, self.s[t]) + self.b_y
#             probability_predicted[t] = softmax(y_predicted[t])
            self.s[t], y_predicted[t], probability_predicted[t] = self._step_forward(x[t], s[t - 1])
            loss += loss_function(probability_predicted[t], y[t])
        return loss, s, y_predicted, probability_predicted 
    
    def backpropagate_loss(self, probability_predicted, x):
        dU, dW, dV = np.zeros_like(self.U), np.zeros_like(self.W), np.zeros_like(self.V)
        db_s, db_y = np.zeros_like(self.b_s), np.zeros_like(self.b_y)
        ds_previous_time = np.zeros_like(self.s[0])
        for t in reversed(range(self.seq_length)):
            # propagate gradients
            probability_predicted_at_t = np.copy(probability_predicted[t])
            d_loss = loss_function_derivative(probability_predicted_at_t, targets[t])
            
            # d(loss)/d(V) = (d(sigmoid(Vs))/d(Vs)) * (d(Vs)/dV) = (d(sigmoid(Vs))/d(Vs)) * s
            dV += np.dot(d_loss, self.s[t].T)
            # d(loss)/d(bias) = (d(sigmoid(Vs))/d(Vs)) * (d(Vs + b)/db) = (d(sigmoid(Vs))/d(Vs)) * 1
            db_y += d_loss
            # d(loss)/d(s) = (d(sigmoid(Vs + b))/d(Vs + b)) * (d(Vs + b)/ds) = (d(sigmoid(Vs))/d(Vs)) * V
            ds = np.dot(self.V.T, d_loss) + ds_previous_time
            
            # d(s)/d(Ws_t-1 + Ux + b) = d(tanh(Ws_t-1 + Ux + b))/d(Ws_t-1 + Ux + b) = (1 - (tanh^2(Ws_t-1 + Ux + b))) * ds
            dsum = (1 - self.s[t] * self.s[t]) * ds
            # d(s)/d(Ws_t-1 + Ux + b) = d(tanh(Ws_t-1 + Ux + b))/d(b) = (1 - (tanh^2(Ws_t-1 + Ux + b))) * 1
            db_s += dsum
            
            # d(sum)/d(U) = d(Ws_t-1 + Ux + b)/dU = x.   => dU = d(next_layer)/dU  * d(next_layer_output)
            dU += np.dot(dsum, x[t].T)
            dW += np.dot(dsum, self.s[t - 1].T)
            ds_previous_time = np.dot(self.V.T, dsum) 
            
        return dU, dW, dV, db_s, db_y, self.s[self.seq_length - 1]
    
    def sample(self, seed, n):
        first_char = np.zeros((vocab_size, 1))
        first_char[seed] = 1
        chars = []
        for t in range(n):
            self._step_forward(first_char, self.s)
            h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
            #compute output (unnormalised)
            y = np.dot(Why, h) + by
            ## probabilities for next chars
            p = np.exp(y) / np.sum(np.exp(y))
            #pick one with the highest probability 
            ix = np.random.choice(range(vocab_size), p=p.ravel())
            #create a vector
            x = np.zeros((vocab_size, 1))
            #customize it for the predicted char
            x[ix] = 1
            #add it to the list
            ixes.append(ix)
        txt = ''.join(ix_to_char[ix] for ix in ixes)
        print '----\n %s \n----' % (txt, )
        
        
        
        hprev = np.zeros((hidden_size,1)) # reset RNN memory  
        #predict the 200 next characters given 'a'
        sample(hprev,char_to_ix['a'],200)
